<a href="https://colab.research.google.com/github/abhyudyasangwan/AI-Course-Designer/blob/main/Course_Designer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Initial Preprocessing**

**Installations**

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.9
    Uninstalling openai-1.59.9:
      Successfully uninstalled openai-1.59.9


In [ ]:
!pip install pandas spacy sentence-transformers scikit-learn
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

**Imports**

In [ ]:
import pandas as pd
from openai import OpenAI
import ast
import re
from sklearn.metrics.pairwise import cosine_similarity
client = OpenAI(api_key='**Hidden**')
df = pd.read_csv('coursera_with_skills.csv')

**Generate Skills List**

In [ ]:
def get_skills_from_gpt4(course_title):
    """
    Sends the course title to GPT-4 and asks for the skills required.
    Returns a comma-separated string of skills.
    """
    prompt = f"""
    You are an expert in analyzing course titles and identifying the skills required to complete the course.
    Based on the course title below, provide a list of skills that a learner must have or will acquire.
    Return only a comma-separated list of skills, nothing else.

    Course Title: {course_title}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Use GPT-4
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,  # Limit the response length
            temperature=0.5  # Control creativity (lower = more deterministic)
        )
        skills = response.choices[0].message.content.strip()
        return skills
    except Exception as e:
        print(f"Error fetching skills for '{course_title}': {e}")
        return None

# Step 4: Loop through each course title and get skills
df['course_skills'] = None  # Add a new column for skills

for index, row in df.iterrows():
    course_title = row['course_title']
    print(f"Processing: {course_title}")

    # Get skills from GPT-4
    skills = get_skills_from_gpt4(course_title)

    # Update the dataset
    df.at[index, 'course_skills'] = skills

# Step 5: Save the updated dataset to a new CSV file
df.to_csv('coursera_with_skills.csv', index=False)
print("Dataset updated and saved as 'coursera_with_skills.csv'.")

Processing: (ISC)² Systems Security Certified Practitioner (SSCP)
Processing: A Crash Course in Causality:  Inferring Causal Effects from Observational Data


KeyboardInterrupt: 

In [ ]:
df

,Unnamed: 0,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,course_skills,Familiarity with (ISC)² Code of Ethics,Incident Response and Recovery,Knowledge of Network and Communications Security,Knowledge of Security Operations and Administration,Understanding of Security Assessment and Testing,Understanding of Risk Identification,Understanding of Systems Security,Understanding of Systems and Application Security,Knowledge of Cryptography,Understanding of Access Controls
0,134,(ISC)² Systems Security Certified Practitioner...,(ISC)²,SPECIALIZATION,4.7,Beginner,5.3k,"Understanding of Systems Security, Knowledge o...",1,1,1,1,1,1,1,1,1,1
1,743,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,COURSE,4.7,Intermediate,17k,None,0,0,0,0,0,0,0,0,0,0
2,874,A Crash Course in Data Science,Johns Hopkins University,COURSE,4.5,Mixed,130k,None,0,0,0,0,0,0,0,0,0,0
3,413,A Law Student's Toolkit,Yale University,COURSE,4.7,Mixed,91k,None,0,0,0,0,0,0,0,0,0,0
4,635,A Life of Happiness and Fulfillment,Indian School of Business,COURSE,4.8,Mixed,320k,None,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,236,Программирование на Python,Mail.Ru Group,SPECIALIZATION,4.5,Intermediate,52k,None,0,0,0,0,0,0,0,0,0,0
887,883,Психолингвистика (Psycholinguistics),Saint Petersburg State University,COURSE,4.8,Mixed,21k,None,0,0,0,0,0,0,0,0,0,0
888,545,Разработка интерфейсов: вёрстка и JavaScript,E-Learning Development Fund,SPECIALIZATION,4.5,Intermediate,30k,None,0,0,0,0,0,0,0,0,0,0
889,875,Русский как иностранный,Saint Petersburg State University,SPECIALIZATION,4.6,Intermediate,9.8k,None,0,0,0,0,0,0,0,0,0,0


**Splitting the various skills into various features**

##**Processing The User Input**

**Takes user input, sends it to GPT-4, and extracts a list of relevant skills needed to solve the issue or achieve the goal. Returns a list of skills.**

In [ ]:
def extract_relevant_skills(user_input):

    prompt = f"""
    You are a career and learning advisor. A user has shared the following issue or goal:

{user_input}

Your task is to:
1) Understand the user's issue or goal.
2) Identify and extract a list of **technical and domain-specific skills** that the user needs to learn or improve to solve the issue or achieve the goal.
3) Exclude general or assumed skills (e.g., critical thinking, communication, problem-solving) that are not specific to the domain.
4) Ensure each skill is listed as a **single, concise term or phrase** (e.g., "Hadoop" instead of "Knowledge of Hadoop & Spark").
5) Return the list of skills in a clear and concise format.

Provide the output as a Python list of skills, like this:
["Skill 1", "Skill 2", "Skill 3"]
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.5
        )

        response_content = response.choices[0].message.content.strip()

        # Extracting the list of skills from the response
        list_match = re.search(r'\[.*\]', response_content)
        if list_match:
            list_string = list_match.group(0)
            try:
                skills_list = eval(list_string)
                if isinstance(skills_list, list):  # Ensuring it's a valid list
                    return skills_list
                else:
                    print("Error: GPT-4 did not return a valid list of skills.")
                    return None
            except Exception as e:
                print(f"Error parsing GPT-4 response into a list: {e}")
                return None
        else:
            print("Error: GPT-4 response does not contain a valid list.")
            return None

    except Exception as e:
        print(f"Error fetching skills from GPT-4: {e}")
        return None

#ENTER UNER INPUT
user_input = str(input("Enter your issue or goal: "))

skills = extract_relevant_skills(user_input)

print("Relevant Skills identified:", skills)

Enter your issue or goal: I want to crack a research internship in artificial intelligence.
Relevant Skills identified: ['Machine Learning', 'Deep Learning', 'Python', 'R', 'Data Mining', 'Natural Language Processing', 'Computer Vision', 'Reinforcement Learning', 'TensorFlow', 'PyTorch', 'Keras', 'SciKit-Learn', 'Statistical Analysis', 'Algorithm Design', 'Big Data Technologies', 'AI Ethics', 'Neural Networks', 'Cloud Computing', 'Distributed Computing', 'High Performance Computing']


**Extracts a Python list from GPT-4's response, even if it includes extra text.**

In [ ]:
def extract_list_from_response(response):
    """
    Extracts a Python list from GPT-4's response, even if it includes extra text.
    """
    try:
        # Finding the start and end of the Python list
        start_index = response.find('[')
        end_index = response.find(']') + 1

        list_str = response[start_index:end_index]

        #Evaluating the string into a Python list
        return eval(list_str)
    except Exception as e:
        print(f"Error extracting list from response: {e}")
        return None

        if isinstance(skills_list, list):
            return skills_list
        else:
            print("Error: GPT-4 did not return a valid list of skills.")
            return None

    except Exception as e:
        print(f"Error fetching skills from GPT-4: {e}")
        return None

**Ask user if they want to modify anything.**

In [ ]:
def final_relevant_skills(user_input, skills):

    prompt = f"""
You are assisting a user who has provided input: "{user_input}" and a list of relevant skills: {skills}. Based on their input, you need to refine the list of relevant skills. Decide which skills to keep, remove, or modify in the list.

Your task is to carefully analyze the user's response and determine whether the list of relevant skills should remain unchanged or if certain skills should be removed or modified. Even if the user does not explicitly state their preferences, infer their intent based on the context, tone, and content of their response.

Ensure that your final output reflects the user's preferences accurately and provides a clear, updated list of relevant skills.

Provide the output as a Python list of skills, like this:
["Skill 1", "Skill 2", "Skill 3"]
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.5
        )
        response_content = response.choices[0].message.content.strip()
        updated_skills = extract_list_from_response(response_content)
        if isinstance(updated_skills, list):
            return updated_skills
        else:
            print("Error: GPT-4 did not return a valid list of skills.")
            return None

    except Exception as e:
        print(f"Error fetching skills from GPT-4: {e}")
        return None

user_input2 = str(input("Is this list of skills to be learnt fine? Mention skills you already know or skills you would like to remove."))

updated_skills = final_relevant_skills(user_input2, skills)

print("Final skills to be learnt:", updated_skills)

Is this list of skills to be learnt fine? Mention skills you already know or skills you would like to remove.I don't want to do R.
Final skills to be learnt: ['Machine Learning', 'Deep Learning', 'Python', 'Data Mining', 'Natural Language Processing', 'Computer Vision', 'Reinforcement Learning', 'TensorFlow', 'PyTorch', 'Keras', 'SciKit-Learn', 'Statistical Analysis', 'Algorithm Design', 'Big Data Technologies', 'AI Ethics', 'Neural Networks', 'Cloud Computing', 'Distributed Computing', 'High Performance Computing']


##**Building Course Recommendation**

In [ ]:
# Splitting the various skills into various features
unique_skills = set()
for skills in df["course_skills"].dropna():
    unique_skills.update(map(str.strip, skills.split(", ")))

unique_skills = list(unique_skills)  # Convert set to list

# Initializing columns with 0
for skill in unique_skills:
    df[skill] = 0

# Marking presence of skills as 1
for index, row in df.iterrows():
    skills_present = set(map(str.strip, row["course_skills"].split(", ")))
    for skill in unique_skills:
        if skill in skills_present:
            df.at[index, skill] = 1

df

AttributeError: 'NoneType' object has no attribute 'split'

**Creating TF-IDF Matrix**

In [ ]:
ds = df.copy()

columns_to_remove = [
    "Unnamed: 0", "course_organization", "course_Certificate_type",
    "course_rating", "course_difficulty", "course_students_enrolled", "course_skills"
]

ds.drop(columns=columns_to_remove, inplace=True, errors="ignore")

from sklearn.feature_extraction.text import TfidfVectorizer

course_titles = ds["course_title"].astype(str).str.lower()

# Initializing the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words="english")

# Computing the matrix
tfidf_matrix = vectorizer.fit_transform(course_titles)

course_title_list = ds["course_title"].tolist()

print("TF-IDF Matrix shape:", tfidf_matrix.shape)

TF-IDF Matrix shape: (891, 1366)


In [ ]:
ds

,course_title,Familiarity with (ISC)² Code of Ethics,Incident Response and Recovery,Knowledge of Network and Communications Security,Knowledge of Security Operations and Administration,Understanding of Security Assessment and Testing,Understanding of Risk Identification,Understanding of Systems Security,Understanding of Systems and Application Security,Knowledge of Cryptography,Understanding of Access Controls
0,(ISC)² Systems Security Certified Practitioner...,1,1,1,1,1,1,1,1,1,1
1,A Crash Course in Causality: Inferring Causal...,0,0,0,0,0,0,0,0,0,0
2,A Crash Course in Data Science,0,0,0,0,0,0,0,0,0,0
3,A Law Student's Toolkit,0,0,0,0,0,0,0,0,0,0
4,A Life of Happiness and Fulfillment,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,Программирование на Python,0,0,0,0,0,0,0,0,0,0
887,Психолингвистика (Psycholinguistics),0,0,0,0,0,0,0,0,0,0
888,Разработка интерфейсов: вёрстка и JavaScript,0,0,0,0,0,0,0,0,0,0
889,Русский как иностранный,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(course_title_list)

['(ISC)² Systems Security Certified Practitioner (SSCP)', 'A Crash Course in Causality:  Inferring Causal Effects from Observational Data', 'A Crash Course in Data Science', "A Law Student's Toolkit", 'A Life of Happiness and Fulfillment', 'ADHD: Everyday Strategies for Elementary Students', 'AI For Everyone', 'AI For Medical Treatment', 'AI Foundations for Everyone', 'AI for Medical Diagnosis', 'AI for Medical Prognosis', 'AI for Medicine', 'AWS Fundamentals', 'AWS Fundamentals: Addressing Security Risk', 'AWS Fundamentals: Building Serverless Applications', 'AWS Fundamentals: Going Cloud-Native', 'AWS Fundamentals: Migrating to the Cloud', 'Aboriginal Worldviews and Education', 'Academic English: Writing', 'Accelerated Computer Science Fundamentals', 'Access Controls', 'Accounting Analytics', 'Accounting for Decision Making', 'Achieving Personal and Professional Success', 'Actualización en el manejo del paciente con diabetes mellitus tipo 2', 'Addiction Treatment: Clinical Skills for

**Converting our list of skills identified into a single string with comma separation**

In [ ]:
skills_string = ', '.join(updated_skills)

final_updated_skills = f'"{skills_string}"'

print(final_updated_skills)

"Machine Learning, Deep Learning, Python, Data Mining, Natural Language Processing, Computer Vision, Reinforcement Learning, TensorFlow, PyTorch, Keras, SciKit-Learn, Statistical Analysis, Algorithm Design, Big Data Technologies, AI Ethics, Neural Networks, Cloud Computing, Distributed Computing, High Performance Computing"


**Finding similar courses using cosine similarity**

In [ ]:
def find_similar_course(input_text, tfidf_matrix, vectorizer, course_title_list, similarity_threshold=0.40):
    input_text = input_text.lower()
    input_vector = vectorizer.transform([input_text])
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix).flatten()

    top_indices = similarity_scores.argsort()[::-1]
    course_list = [course_title_list[i] for i in top_indices if similarity_scores[i] > similarity_threshold]

    return course_list

final_course_list = find_similar_course(final_updated_skills, tfidf_matrix, vectorizer, course_title_list)

print("Courses found:", final_course_list)

Courses found: ['Cloud Computing', 'Introduction to Cloud Computing', 'Fundamentals of Computing']


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight BERT model

def find_similar_course_bert(input_text, course_title_list, similarity_threshold=0.40):
    input_text = input_text.lower()
    input_embedding = model.encode([input_text])
    course_embeddings = model.encode(course_title_list)

    similarity_scores = cosine_similarity(input_embedding, course_embeddings).flatten()
    course_list = [course_title_list[i] for i, score in enumerate(similarity_scores) if score > similarity_threshold]

    return course_list

final_course_list = find_similar_course_bert(final_updated_skills, course_title_list)
print("Courses found (BERT):", final_course_list)

Courses found (BERT): ['AI For Everyone', 'AI Foundations for Everyone', 'Advanced Machine Learning with TensorFlow on Google Cloud Platform', 'Applied Machine Learning in Python', 'Data Engineering, Big Data, and Machine Learning on GCP', 'Data Science', 'Machine Learning for All', 'Machine Learning with Python', 'Machine Learning with TensorFlow on Google Cloud Platform', 'Python for Data Science and AI', 'Structuring Machine Learning Projects', 'What is Data Science?']


**REMOVING OVERLAPPING COURSES**

In [ ]:
def extract_relevant_skills2(final_course_list):

    prompt = f"""
    You are an expert in analyzing and optimizing course lists. I have a list of courses {final_course_list}, which contains multiple courses. Each course has a title. Your task is to:

    1) Analyze the courses in final_course_list and identify any overlapping content between them.

    2) For courses with significant overlap, determine which course covers the majority of the content and retain that one. Remove the others to avoid redundancy.

    3) Ensure that any unique courses (those without significant overlap) are kept in the list.

    4) Return a refined and optimized version of final_course_list that minimizes redundancy while preserving all unique and essential content.

    5) Provide the final list of courses in a clear and organized format.

    **Important:** Your output must be a Python list of course titles, like this, always in this format:
    ["Course Title 1", "Course Title 2", "Course Title 3"]

    **Do not include any additional text or explanations in your response. Only return the Python list.**
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.5
        )

        response_content = response.choices[0].message.content.strip()

        try:
            response_content = response_content.strip()
            if response_content.startswith("```python"):
                response_content = response_content[len("```python"):].strip()
            if response_content.endswith("```"):
                response_content = response_content[:-len("```")].strip()

            skills_list = ast.literal_eval(response_content)
            if isinstance(skills_list, list):
                return skills_list
            else:
                print("Error: GPT-4 did not return a valid list of skills.")
                return None
        except Exception as e:
            print(f"Error parsing GPT-4 response into a list: {e}")
            print("Raw Response Content:", response_content)
            return None

    except Exception as e:
        print(f"Error fetching skills from GPT-4: {e}")
        return None

final_response_list = extract_relevant_skills2(final_course_list)

print("Optimized Course List:", final_response_list)

Optimized Course List: ['Machine Learning and Reinforcement Learning in Finance', 'Natural Language Processing in TensorFlow', 'Data Analysis with Python', 'Modern Big Data Analysis with SQL', 'Deep Learning', 'Machine Learning for All', 'Google Cloud Platform Big Data and Machine Learning Fundamentals', 'Data Engineering, Big Data, and Machine Learning on GCP', 'Practical Time Series Analysis', 'Fundamentals of Reinforcement Learning', 'Data Science: Statistics and Machine Learning']


##**Final Output Processing**

In [ ]:
def sort_courses(final_response_list):

    prompt = f"""
    You are an expert career and learning advisor. A user has provided a list of courses they need to complete, and your task is to analyze and sort these courses in the order they should be completed. The goal is to create a logical learning path that considers the following factors:

1. **Difficulty Level:** Easier courses should generally come before more advanced ones.
2. **Prerequisites:** Courses that build on foundational knowledge should come after those that teach the basics.
3. **Logical Progression:** Courses should flow naturally from one to the next, ensuring the user gains skills in a structured way.

Here is the list of courses:
{final_course_list}

Your task is:
1) Analyze the courses and determine the optimal order in which the user should complete them.
2) Return the updated `final_response_list` with the courses sorted in the recommended order.

**Important:**
- Your output must be **only a Python list of course titles**, sorted in the recommended order, like this:
["Course Title 1", "Course Title 2", "Course Title 3"]

- **Do not include any additional text, explanations, or numbering in your response. Only return the Python list.**
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.5
        )

        response_content = response.choices[0].message.content.strip()

        optimized_course_list = extract_list_from_response(response_content)
        if isinstance(optimized_course_list, list):
            return optimized_course_list
        else:
            print("Error: GPT-4 did not return a valid list of skills.")
            return None

    except Exception as e:
        print(f"Error fetching skills from GPT-4: {e}")
        return None

final_response_list = sort_courses(final_response_list)

print("Optimized Course List:", final_response_list)

Optimized Course List: ['Data Analysis with Python', 'Modern Big Data Analysis with SQL', 'Google Cloud Platform Big Data and Machine Learning Fundamentals', 'Data Engineering, Big Data, and Machine Learning on GCP', 'Practical Time Series Analysis', 'Data Science: Statistics and Machine Learning', 'Machine Learning for All', 'Machine Learning with Python', 'Machine Learning', 'Machine Learning with TensorFlow on Google Cloud Platform', 'Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning', 'Deep Learning', 'Natural Language Processing in TensorFlow', 'Fundamentals of Reinforcement Learning', 'Reinforcement Learning', 'Machine Learning and Reinforcement Learning in Finance']


In [ ]:
def generate_course_details(final_response_list, df):
    result_list = []

    for course_title in final_response_list:
        matched_row = df[df['course_title'] == course_title]

        if not matched_row.empty:
            course_details = [
                matched_row['course_title'].values[0],
                matched_row['course_organization'].values[0],
                matched_row['course_rating'].values[0],
                matched_row['course_difficulty'].values[0],
            ]

            result_list.append(course_details)
        else:
            print(f"Warning: Course title '{course_title}' not found in the DataFrame.")

    return result_list

course_details_list = generate_course_details(final_response_list, df)

print("Generated Course Details:")
for course in course_details_list:
    print(course)

Generated Course Details:
['Data Analysis with Python', 'IBM', 4.7, 'Beginner']
['Modern Big Data Analysis with SQL', 'Cloudera', 4.8, 'Beginner']
['Google Cloud Platform Big Data and Machine Learning Fundamentals', 'Google Cloud', 4.6, 'Intermediate']
['Data Engineering, Big Data, and Machine Learning on GCP', 'Google Cloud', 4.6, 'Intermediate']
['Practical Time Series Analysis', 'The State University of New York', 4.6, 'Intermediate']
['Data Science: Statistics and Machine Learning', 'Johns Hopkins University', 4.4, 'Intermediate']
['Machine Learning for All', 'University of London', 4.6, 'Beginner']
['Machine Learning with Python', 'IBM', 4.7, 'Intermediate']
['Machine Learning', 'University of Washington', 4.6, 'Intermediate']
['Machine Learning with TensorFlow on Google Cloud Platform', 'Google Cloud', 4.5, 'Intermediate']
['Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning', 'deeplearning.ai', 4.7, 'Intermediate']
['Deep Learning', 'deep

In [ ]:
for course in course_details_list:
    print("-------------")
    print(f"Course Name: {course[0]}")
    print(f"Company Offering: {course[1]}")
    print(f"Rating: {course[2]}")
    print(f"Level: {course[3]}")
    prompt = f"""
    The user has given a user input {user_input}. For this you need to write how {course[0]} will help achieve what the user is aiming for. Don't give unnecesary response detials. Simply give the answer. Just a small paragraph.
    """
        # Send the prompt to GPT-4
    response = client.chat.completions.create(
    model="gpt-4",  # Use GPT-4
    messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,  # Adjust as needed
            temperature=0.5  # Control creativity (lower = more deterministic)
    )

        # Extract the response content
    response_content = response.choices[0].message.content.strip()

        # Debugging: Print the raw response to check for issues
    print(response_content)
    print("-------------\n")

-------------
Course Name: Data Analysis with Python
Company Offering: IBM
Rating: 4.7
Level: Beginner
Data Analysis with Python is crucial for an AI research internship as it provides the necessary skills to process, analyze and interpret complex data which is fundamental in AI. Python is a versatile language with libraries like Pandas, NumPy, and Matplotlib that are specifically designed for data analysis. These help in cleaning, visualizing, and understanding data, which are essential in building effective AI models. Furthermore, machine learning, a subset of AI, relies heavily on statistical analysis and predictive modeling, both of which are facilitated by Python. Hence, proficiency in data analysis with Python can significantly enhance your research capabilities in artificial intelligence.
-------------

-------------
Course Name: Modern Big Data Analysis with SQL
Company Offering: Cloudera
Rating: 4.8
Level: Beginner
Modern Big Data Analysis with SQL is crucial for a research in